In [2]:
import numpy as np
from sklearn.datasets import make_classification
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import SGD

np.random.seed(42)

# Generate simulated dataset...
X, y = make_classification(n_samples=10000, n_features=10, n_classes=2, random_state=42)

# Split the data into 3 parts
split1 = int(len(X) / 3)
split2 = int(2 * len(X) / 3)
X_train1, y_train1 = X[:split1], y[:split1]
X_train2, y_train2 = X[split1:split2], y[split1:split2]
X_test, y_test = X[split2:], y[split2:]

# Global LSTM RNN model
global_model = Sequential()
global_model.add(LSTM(32, input_shape=(X.shape[1], 1)))
global_model.add(Dense(1, activation='sigmoid'))

# Compile the global model
global_model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.01), metrics=['accuracy'])

# Let's train the global model on all data
global_model.fit(X.reshape(-1, X.shape[1], 1), y, epochs=10, batch_size=32, verbose=0)

# Define the local LSTM RNN model1
local_model1 = Sequential()
local_model1.add(LSTM(32, input_shape=(X.shape[1], 1)))
local_model1.add(Dense(1, activation='sigmoid'))

# Compile the local model1
local_model1.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.01), metrics=['accuracy'])

# Define the local LSTM RNN model2
local_model2 = Sequential()
local_model2.add(LSTM(32, input_shape=(X.shape[1], 1)))
local_model2.add(Dense(1, activation='sigmoid'))

# Compile the local model2
local_model2.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.01), metrics=['accuracy'])

# Simulate local training and updating of global model
num_rounds = 10
for i in range(num_rounds):
    # Get local model weights and send to global model
    local_weights1 = local_model1.get_weights()
    local_weights2 = local_model2.get_weights()
    avg_weights = (0.4 * np.array(local_weights1) + 0.6 * np.array(local_weights2))## weighted aggregation

    global_model.set_weights(avg_weights)

    # Evaluate global model on test dataset
    loss, acc = global_model.evaluate(X_test.reshape(-1, X_test.shape[1], 1), y_test, verbose=0)
    print(f'Round {i+1} - Global Model: Accuracy={acc:.4f}')

    # Train local model1 on local dataset
    local_model1.fit(X_train1.reshape(-1, X_train1.shape[1], 1), y_train1, epochs=3, batch_size=32, verbose=0)

    # Train local model2 on local dataset
    local_model2.fit(X_train2.reshape(-1, X_train2.shape[1], 1), y_train2, epochs=3, batch_size=32, verbose=0)


<ipython-input-2-0bd282690d22>:52: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  avg_weights = (0.4 * np.array(local_weights1) + 0.6 * np.array(local_weights2))


Round 1 - Global Model: Accuracy=0.3104
Round 2 - Global Model: Accuracy=0.6560
Round 3 - Global Model: Accuracy=0.6842
Round 4 - Global Model: Accuracy=0.7118
Round 5 - Global Model: Accuracy=0.7376
Round 6 - Global Model: Accuracy=0.7460
Round 7 - Global Model: Accuracy=0.7522
Round 8 - Global Model: Accuracy=0.7519
Round 9 - Global Model: Accuracy=0.7558
Round 10 - Global Model: Accuracy=0.7585
